In [1]:
from picai_baseline.splits.picai import train_splits, valid_splits

If you have questions or suggestions, feel free to open an issue at https://github.com/DIAGNijmegen/picai_baseline



In [2]:
for fold, ds_config in train_splits.items():
    print(f"Training fold {fold} has cases: {ds_config['subject_list']}")

Training fold 0 has cases: ['10002_1000002', '10004_1000004', '10005_1000005', '10007_1000007', '10008_1000008', '10009_1000009', '10010_1000010', '10011_1000011', '10012_1000012', '10013_1000013', '10014_1000014', '10015_1000015', '10016_1000016', '10018_1000018', '10019_1000019', '10021_1000021', '10024_1000024', '10025_1000025', '10026_1000026', '10028_1000028', '10030_1000030', '10031_1000031', '10034_1000034', '10036_1000036', '10037_1000037', '10039_1000039', '10042_1000042', '10043_1000043', '10044_1000044', '10046_1000046', '10047_1000047', '10048_1000048', '10049_1000049', '10050_1000050', '10051_1000051', '10052_1000052', '10053_1000053', '10054_1000054', '10055_1000055', '10056_1000056', '10057_1000057', '10058_1000058', '10060_1000060', '10061_1000061', '10063_1000063', '10064_1000064', '10066_1000066', '10067_1000067', '10068_1000068', '10069_1000069', '10070_1000070', '10072_1000072', '10074_1000074', '10075_1000075', '10076_1000076', '10077_1000077', '10079_1000079', '10

In [3]:
for fold, ds_config in valid_splits.items():
    print(f"Validation fold {fold} has cases: {ds_config['subject_list']}")

Validation fold 0 has cases: ['10000_1000000', '10001_1000001', '10003_1000003', '10006_1000006', '10017_1000017', '10020_1000020', '10022_1000022', '10023_1000023', '10027_1000027', '10029_1000029', '10032_1000032', '10033_1000033', '10035_1000035', '10038_1000038', '10040_1000040', '10041_1000041', '10045_1000045', '10059_1000059', '10062_1000062', '10065_1000065', '10071_1000071', '10073_1000073', '10078_1000078', '10080_1000080', '10082_1000082', '10083_1000083', '10085_1000085', '10092_1000092', '10109_1000109', '10129_1000129', '10129_1000130', '10136_1000138', '10142_1000144', '10146_1000148', '10156_1000159', '10158_1000161', '10165_1000168', '10167_1000170', '10173_1000176', '10177_1000180', '10181_1000184', '10189_1000192', '10193_1000196', '10193_1000197', '10195_1000199', '10196_1000200', '10197_1000201', '10198_1000202', '10229_1000233', '10237_1000241', '10243_1000247', '10257_1000261', '10263_1000267', '10266_1000270', '10272_1000276', '10282_1000288', '10294_1000300', '

In [4]:
from picai_baseline.splits.picai_nnunet import train_splits, valid_splits

In [1]:
import sys
from pathlib import Path
from collections import defaultdict

import nibabel as nib   # pip install nibabel
import numpy as np
import pandas as pd

mask_dir = "input/picai_labels/csPCa_lesion_delineations/human_expert/resampled"

mask_dir = Path(mask_dir)
if not mask_dir.exists():
    raise FileNotFoundError(f"{mask_dir} does not exist")

total_counts = defaultdict(int)

for mask_path in mask_dir.glob("*.nii.gz"):
    mask = nib.load(str(mask_path)).get_fdata()   # float64 by default
    unique, counts = np.unique(mask, return_counts=True)
    print(f"Processing {mask_path.name}: found {unique} unique pixel values {counts}")

    for u, c in zip(unique, counts):

        # Cast to int if labels are integer-encoded; adjust if your labels are floats
        total_counts[int(u)] += int(c)

# ---------- save to CSV ----------
pixel_values = sorted(total_counts.keys())
counts_row   = [total_counts[pv] for pv in pixel_values]

df = pd.DataFrame([counts_row], columns=pixel_values)
df.to_csv("mask_pixel_frequencies.csv", index=False)

print("✅ Saved pixel-frequency table to mask_pixel_frequencies.csv")



Processing 10000_1000000.nii.gz: found [0.] unique pixel values [12697600]
Processing 10001_1000001.nii.gz: found [0.] unique pixel values [8601600]
Processing 10002_1000002.nii.gz: found [0.] unique pixel values [3244032]
Processing 10003_1000003.nii.gz: found [0.] unique pixel values [3391488]
Processing 10004_1000004.nii.gz: found [0.] unique pixel values [8601600]
Processing 10005_1000005.nii.gz: found [0. 3.] unique pixel values [2801186     478]
Processing 10006_1000006.nii.gz: found [0.] unique pixel values [3096576]
Processing 10007_1000007.nii.gz: found [0.] unique pixel values [7782400]
Processing 10009_1000009.nii.gz: found [0.] unique pixel values [28311552]
Processing 10010_1000010.nii.gz: found [0.] unique pixel values [7782400]
Processing 10011_1000011.nii.gz: found [0.] unique pixel values [3981312]
Processing 10012_1000012.nii.gz: found [0. 2.] unique pixel values [3390062    1426]
Processing 10014_1000014.nii.gz: found [0.] unique pixel values [9011200]
Processing 100

In [ ]:

import sys
from pathlib import Path

import nibabel as nib          # pip install nibabel
import numpy as np
import pandas as pd


PIXEL_VALUES = list(range(6))   # 0,1,2,3,4,5

mask_dir = "input/picai_labels/csPCa_lesion_delineations/human_expert/resampled"
mask_dir = Path(mask_dir)
if not mask_dir.exists():
    raise FileNotFoundError(f"{mask_dir} does not exist")

rows = []
for mask_path in sorted(mask_dir.glob("*.nii.gz")):
    vol = nib.load(str(mask_path)).get_fdata()
    # cast to int in case the mask is float‐encoded
    counts = np.bincount(vol.astype(int).ravel(), minlength=6)[:6]

    rows.append(
        {"mask": mask_path.stem, **{v: int(counts[v]) for v in PIXEL_VALUES}}
    )
    print(f"Processed {mask_path.stem} with counts: {counts}")

df = pd.DataFrame(rows, columns=["mask"] + PIXEL_VALUES)
df.to_csv("mask_pixel_frequencies_all.csv", index=False)
print("✅ Saved frequencies to mask_pixel_frequencies_all.csv")


In [9]:
import pandas as pd

df = pd.read_csv("mask_pixel_frequencies_all.csv")

# sum of all pixel values for each mask

for i in range(6):
    print(i, df[df[str(i)] > 0].shape[0], "cases with >0 pixels")


0 1295 cases with >0 pixels
1 0 cases with >0 pixels
2 135 cases with >0 pixels
3 52 cases with >0 pixels
4 20 cases with >0 pixels
5 18 cases with >0 pixels
